In [22]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, MaxPooling2D
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import Conv2D, BatchNormalization, GlobalAveragePooling2D, Flatten, Dropout, Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model

In [2]:
TRAIN_IMAGE_PATH = "jpeg/train/"
TEST_IMAGE_PATH = "jpeg/test/"
IMG_HEIGHT = 500
IMG_WIDTH = 500
BATCH_SIZE = 64
AUTO = tf.data.experimental.AUTOTUNE
REG = 0.0005
EPOCHS=20
METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
]

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [4]:
train_df["image_name"] = train_df["image_name"].apply(lambda x: TRAIN_IMAGE_PATH + x + ".jpg")
test_df["image_name"] = test_df["image_name"].apply(lambda x: TEST_IMAGE_PATH + x + ".jpg")

In [5]:
# train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=45, shuffle=True)

In [6]:
# train_df.shape, val_df.shape

In [7]:
def decode_image(filename, label=None, image_size=(IMG_WIDTH, IMG_HEIGHT)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32)/255.0
    image = tf.image.resize(image, size=image_size)
    
    if label is None:
        return image
    else:
        return image, label

In [8]:
def data_augment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    if label is None:
        return image
    else:
        return image, label

In [9]:
# def compute_class_weights(labels):
#     total_labels = labels.shape[0]
    
#     positive_labels = K.sum(labels, axis=0)/total_labels
#     negative_labels = 1 - positive_labels
#     return {0:positive_labels, 1:negative_labels}

In [10]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
image_names = train_df["image_name"].values
train_df["image_name"] = lb.fit_transform(train_df["image_name"].values)
train_df["target"] = train_df["target"].astype("int")
train_df.head()
map_name_no = dict(zip(train_df["image_name"], image_names))
y_train = train_df["target"]
x_train = train_df[["image_name"]]


over = SMOTE(random_state=45, sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('o', over), ('u', under)]
ppl = Pipeline(steps=steps)

x_train, y_train = ppl.fit_resample(x_train, y_train)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=45)
x_train["image_name"] = x_train["image_name"].apply(lambda x: map_name_no[x])
x_val["image_name"] = x_val["image_name"].apply(lambda x: map_name_no[x])

Using TensorFlow backend.


In [11]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

In [12]:
class_weights = dict(enumerate(class_weights))

In [13]:
class_weights

{0: 0.752022342064715, 1: 1.491975544516622}

In [14]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

((7809, 1), (7809,), (1953, 1), (1953,))

In [15]:
sum(y_train), sum(y_val)

(2617, 637)

In [16]:
train_dataset = (tf.data.Dataset
                 .from_tensor_slices((x_train["image_name"].values, y_train))
                 .map(decode_image, num_parallel_calls=AUTO)
                 .map(data_augment, num_parallel_calls=AUTO)
                 .repeat()
                 .shuffle(512)
                 .batch(BATCH_SIZE)
                 .prefetch(AUTO)
                )

In [17]:
val_dataset = (tf.data.Dataset
               .from_tensor_slices((x_val["image_name"].values, y_val))
               .map(decode_image, num_parallel_calls=AUTO)
               .map(data_augment, num_parallel_calls=AUTO)
               .repeat()
               .shuffle(512)
               .batch(BATCH_SIZE)
               .prefetch(AUTO))

In [18]:
test_dataset = (tf.data.Dataset.from_tensor_slices((test_df.image_name))
                .map(decode_image, num_parallel_calls=AUTO)
                .batch(BATCH_SIZE))

In [19]:
lr = ReduceLROnPlateau(
    monitor="val_loss",
    patience=3,
    min_lr=0.000001,
    factor=0.5,
    verbose=1
)

In [20]:
es = EarlyStopping(monitor="val_loss", patience=10)

In [23]:
model_d201 = DenseNet201(include_top=False, weights="imagenet", input_shape=(IMG_WIDTH, IMG_HEIGHT,3))
for layers in model_d201.layers[:-10]:
    layers.trainable = False
model_d201.trainable=False

model = Conv2D(filters=32, kernel_size=(3,3), data_format="channels_last", activation="relu", kernel_regularizer=l2(REG))(model_d201.output)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)
model = Conv2D(filters=32, kernel_size=(3,3), activation="relu", kernel_regularizer=l2(REG))(model)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)
model = MaxPooling2D()(model)
model = Dropout(0.25)(model)

model = Conv2D(filters=64, kernel_size=(5,5), data_format="channels_last", activation="relu", kernel_regularizer=l2(REG))(model_d201.output)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)
model = Conv2D(filters=64, kernel_size=(5,5), activation="relu", kernel_regularizer=l2(REG))(model)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)
model = MaxPooling2D()(model)
model = Dropout(0.25)(model)


model = Flatten()(model)
model = Dense(256, activation="relu")(model)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)
model = Dropout(0.5)(model)
model = Dense(64, activation="relu")(model)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)

output = Dense(1, activation="sigmoid")(model)
model_d201 = Model(inputs=model_d201.input, outputs=output)
model_d201.summary()
model_d201.compile(optimizer=tf.keras.optimizers.RMSprop(lr=1e-4), loss="binary_crossentropy", metrics=METRICS)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 500, 500, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 506, 506, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 250, 250, 64) 9408        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 250, 250, 64) 256         conv1/conv[0][0]                 
______________________________________________________________________________________________

conv5_block3_0_relu (Activation (None, 15, 15, 960)  0           conv5_block3_0_bn[0][0]          
__________________________________________________________________________________________________
conv5_block3_1_conv (Conv2D)    (None, 15, 15, 128)  122880      conv5_block3_0_relu[0][0]        
__________________________________________________________________________________________________
conv5_block3_1_bn (BatchNormali (None, 15, 15, 128)  512         conv5_block3_1_conv[0][0]        
__________________________________________________________________________________________________
conv5_block3_1_relu (Activation (None, 15, 15, 128)  0           conv5_block3_1_bn[0][0]          
__________________________________________________________________________________________________
conv5_block3_2_conv (Conv2D)    (None, 15, 15, 32)   36864       conv5_block3_1_relu[0][0]        
__________________________________________________________________________________________________
conv5_bloc

In [ ]:
x_val.shape

In [26]:
history = model_d201.fit(train_dataset, epochs=EPOCHS, callbacks=[lr, es],
                        steps_per_epoch=x_train.shape[0]//BATCH_SIZE, validation_data=val_dataset,
                        validation_steps=x_val.shape[0]//BATCH_SIZE,
                        class_weight = class_weights)

Train for 122 steps, validate for 30 steps
Epoch 1/20
122/122 [==============================] - 1015s 8s/step - loss: 0.9416 - tp: 1381.0000 - fp: 2576.0000 - tn: 2623.0000 - fn: 1228.0000 - accuracy: 0.5128 - precision: 0.3490 - recall: 0.5293 - auc: 0.5179 - val_loss: 0.8320 - val_tp: 39.0000 - val_fp: 63.0000 - val_tn: 1245.0000 - val_fn: 573.0000 - val_accuracy: 0.6687 - val_precision: 0.3824 - val_recall: 0.0637 - val_auc: 0.5100
Epoch 2/20
122/122 [==============================] - 1059s 9s/step - loss: 0.8589 - tp: 1437.0000 - fp: 2544.0000 - tn: 2658.0000 - fn: 1169.0000 - accuracy: 0.5245 - precision: 0.3610 - recall: 0.5514 - auc: 0.5434 - val_loss: 0.8115 - val_tp: 92.0000 - val_fp: 153.0000 - val_tn: 1140.0000 - val_fn: 535.0000 - val_accuracy: 0.6417 - val_precision: 0.3755 - val_recall: 0.1467 - val_auc: 0.5089
Epoch 3/20
122/122 [==============================] - 1088s 9s/step - loss: 0.8105 - tp: 1507.0000 - fp: 2374.0000 - tn: 2800.0000 - fn: 1127.0000 - accuracy: 0.5

121/122 [============================>.] - ETA: 4s - loss: 0.5955 - tp: 1945.0000 - fp: 1486.0000 - tn: 3662.0000 - fn: 651.0000 - accuracy: 0.7240 - precision: 0.5669 - recall: 0.7492 - auc: 0.8032
Epoch 00019: ReduceLROnPlateau reducing learning rate to 6.24999984211172e-06.
122/122 [==============================] - 1013s 8s/step - loss: 0.5953 - tp: 1965.0000 - fp: 1495.0000 - tn: 3693.0000 - fn: 655.0000 - accuracy: 0.7246 - precision: 0.5679 - recall: 0.7500 - auc: 0.8036 - val_loss: 0.8248 - val_tp: 98.0000 - val_fp: 212.0000 - val_tn: 1096.0000 - val_fn: 514.0000 - val_accuracy: 0.6219 - val_precision: 0.3161 - val_recall: 0.1601 - val_auc: 0.4837
Epoch 20/20
122/122 [==============================] - 1033s 8s/step - loss: 0.5738 - tp: 2010.0000 - fp: 1460.0000 - tn: 3740.0000 - fn: 598.0000 - accuracy: 0.7364 - precision: 0.5793 - recall: 0.7707 - auc: 0.8233 - val_loss: 0.8065 - val_tp: 160.0000 - val_fp: 344.0000 - val_tn: 960.0000 - val_fn: 456.0000 - val_accuracy: 0.5833 -

In [27]:
probs = model_d201.predict(test_dataset, verbose = 1)

172/172 [==============================] - 642s 4s/step


In [28]:
sam = pd.read_csv("sample_submission.csv")

In [29]:
sam.head()


,image_name,target
0,ISIC_0052060,0
1,ISIC_0052349,0
2,ISIC_0058510,0
3,ISIC_0073313,0
4,ISIC_0073502,0


In [30]:
sam["target"] = probs

In [31]:
sam

,image_name,target
0,ISIC_0052060,0.219937
1,ISIC_0052349,0.489566
2,ISIC_0058510,0.491473
3,ISIC_0073313,0.418109
4,ISIC_0073502,0.183890
...,...,...
10977,ISIC_9992485,0.359625
10978,ISIC_9996992,0.524004
10979,ISIC_9997917,0.378495
10980,ISIC_9998234,0.267345


In [32]:
sam.to_csv("model_d201_smote_oversampling_undersampling.csv", index=False)